In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key


In [2]:
#Import weather data

weather_df = pd.read_csv("..\\WeatherPy\\weather_data.csv")
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hemisphere
0,Half Moon Bay,37.4636,-122.4286,57.78,71,1,3.44,Northern
1,Nikolskoye,59.7035,30.7861,45.10,87,90,6.71,Northern
2,Torbay,47.6666,-52.7314,31.28,64,75,28.77,Northern
3,Peniche,39.3558,-9.3811,72.00,56,92,11.99,Northern
4,Novy Urengoy,66.0833,76.6333,-22.99,99,5,6.38,Northern


In [3]:
#Store variables to be used in map

locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [4]:
#Create map
gmaps.configure(api_key=g_key)

figure = gmaps.figure()
humidity_heatmap = gmaps.heatmap_layer(locations, weights=humidity, 
                                       dissipating=False, max_intensity=100, 
                                       point_radius = 2.5)

figure.add_layer(humidity_heatmap)

figure

Figure(layout=FigureLayout(height='420px'))

In [5]:
humidity

0      71.0
1      87.0
2      64.0
3      56.0
4      99.0
       ... 
501    99.0
502    93.0
503    84.0
504    33.0
505    44.0
Name: Humidity, Length: 506, dtype: float64

In [6]:
#Narrow down dataframe

vacation_df = weather_df.loc[(weather_df["Temperature"] < 85) & (weather_df["Temperature"] > 70) & (weather_df["Wind_Speed"] < 10) & (weather_df["Cloudiness"] < 10)]
vacation_df = vacation_df.dropna()
vacation_df.count()

City           19
Latitude       19
Longitude      19
Temperature    19
Humidity       19
Cloudiness     19
Wind_Speed     19
Hemisphere     19
dtype: int64

In [7]:
coords_list = []
for index, row in vacation_df.iterrows():
    coords = str(row['Latitude']) + "," + str(row['Longitude'])
    coords_list.append(coords)


vacation_df["Locations"] = coords_list
coords_list

['-20.9373,55.2919',
 '-23.7781,-45.3581',
 '25.1216,62.3254',
 '25.2582,55.3047',
 '-30.0319,-70.7081',
 '-23.1333,150.7333',
 '17.0151,54.0924',
 '-38.9516,-68.0591',
 '-34.6515,-59.4307',
 '14.8961,-24.4956',
 '-12.0432,-77.0282',
 '-8.2578,-49.2647',
 '22.5667,59.5289',
 '17.4189,121.4443',
 '18.6853,12.9164',
 '4.9158,29.4742',
 '25.3,73.9333',
 '21.4267,39.8261',
 '27.6652,54.1371']

In [8]:
#Work with Google APIs to get hotels

hotels_list = []
new_coords = []
for index, row in vacation_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": row["Locations"],
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    response = requests.get(base_url, params)
    full = response.json()
    hotels = response.json()["results"]
    if hotels:
        hotels_list.append(hotels[0]["name"])
        new_coords.append(row["Locations"])

hotels

[]

In [9]:
hotel_df = pd.DataFrame({"Hotel Name": hotels_list, "Locations": new_coords})
hotel_df = pd.merge(hotel_df, vacation_df, on='Locations', how='left')
hotel_df

,Hotel Name,Locations,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hemisphere
0,Los Mosaicos,"-30.0319,-70.7081",Vicuña,-30.0319,-70.7081,77.0,28,0,1.01,Southern


In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

info_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

figure.add_layer(info_layer)

In [11]:
figure

Figure(layout=FigureLayout(height='420px'))